### <img src="https://storage.googleapis.com/data-analytics-golden-demo/colab-diagrams/BigQuery-32-color.svg" width="30" valign="top" alt="BigQuery"> <img src="https://storage.googleapis.com/data-analytics-golden-demo/colab-diagrams/CloudSQL-32-color.svg" width="30" valign="top" alt="AlloyDB"> <img src="https://storage.googleapis.com/data-analytics-golden-demo/colab-diagrams/VertexAI-32-color.svg" width="30" valign="top" alt="VertexAI"> Architecture

In [ ]:
# Architecture Diagram
from IPython.display import Image
Image(url='https://storage.googleapis.com/data-analytics-golden-demo/colab-diagrams/Architecture-Data-Engineering-Agents.png', width=1000)

### <img src="https://storage.googleapis.com/data-analytics-golden-demo/colab-diagrams/BigQuery-32-color.svg" width="30" valign="top" alt="BigQuery"> View Raw Unprocessed Competitor Pricing data

In [ ]:
%%bigquery
----------------------------------------------------------------------------------------------------------------
-- Malformed data, Hotel name is not proper case, phone has various formats
----------------------------------------------------------------------------------------------------------------
SELECT * FROM `data-connect-demo2.raw_data.competitor_pricing`;

### <img src="https://storage.googleapis.com/data-analytics-golden-demo/colab-diagrams/BigQuery-32-color.svg" width="30" valign="top" alt="BigQuery"> Data Engineering Agents

In [ ]:
%%bigquery
----------------------------------------------------------------------------------------------------------------
-- Clean up, drop existing table in case we want schema changes
----------------------------------------------------------------------------------------------------------------
DROP TABLE IF EXISTS data-connect-demo2.cleaned_data.pricing_1;

**Prompts**
1. Source and Destinations:
   - I have a table competitor_pricing in dataset raw_data.  I want to make the hotel name, room type and bed type columns to be lowercase and the first letter of each word uppercase.  I want to save the data to a table named pricing_1 in dataset cleaned_data.

2. Cleaning phone number
   - I want the phone number field to only contain numbers.

3. Cleaning dates
   - In the date field replace forward slashes with a dash.

4. Cleaning data type
   - I want the room_size_sqft and price columns to be numeric data types in the destination table.  Also make the date field a DATE datatype.

In [ ]:
%%bigquery
----------------------------------------------------------------------------------------------------------------
-- Show the imported / cleaned data from the Data Engineering Agent
----------------------------------------------------------------------------------------------------------------
SELECT *
  FROM  data-connect-demo2.cleaned_data.pricing;

### <img src="https://storage.googleapis.com/data-analytics-golden-demo/colab-diagrams/BigQuery-32-color.svg" width="30" valign="top" alt="BigQuery"> Gemini and Text Embeddings

In [ ]:
%%bigquery
----------------------------------------------------------------------------------------------------------------
-- Create a pricing table with a primary key and create our Gemini and Text Embedding models
----------------------------------------------------------------------------------------------------------------
CREATE TABLE IF NOT EXISTS `data-connect-demo2.cleaned_data.pricing` AS
SELECT ROW_NUMBER() OVER (PARTITION BY 1) AS pricing_id, *
  FROM `data-connect-demo2.cleaned_data.pricing_1`;

-- Create our GenAI and Vector Embeddings models
CREATE MODEL IF NOT EXISTS `data-connect-demo2.cleaned_data.gemini_2_0_flash`
  REMOTE WITH CONNECTION `data-connect-demo2.us-central1.vertex-ai`
  OPTIONS (endpoint = 'gemini-2.0-flash');

CREATE MODEL IF NOT EXISTS `data-connect-demo2.cleaned_data.text_embedding_005`
  REMOTE WITH CONNECTION `data-connect-demo2.us-central1.vertex-ai`
  OPTIONS (endpoint = 'text-embedding-005');

We now want to create embeddings to match semantically similar items:

- Balcony/Terrace/Veranda/Sundeck/Porch: (These are all open-air platforms attached to the room)
- Mini-fridge/Refrigerator/Fridge: (Different names for the same appliance)
- Working Desk/Writing Table/Desk: (All the same thing - a surface for working)
- Down Pillows and Duvets: (Specific type of bedding)


In [ ]:
%%bigquery
----------------------------------------------------------------------------------------------------------------
-- Create embeddings using text-embedding-005 on each room feature (we split it by the pipe)
----------------------------------------------------------------------------------------------------------------
CREATE TABLE IF NOT EXISTS `data-connect-demo2.cleaned_data.pricing_embeddings` AS
WITH split_room_features AS
(
  SELECT pricing_id, SPLIT(LOWER(room_features), '|') AS room_features_array
    FROM `cleaned_data.pricing`
),
room_features AS
(
  SELECT pricing_id, room_feature
    FROM split_room_features
         JOIN UNNEST(room_features_array) AS room_feature
)
SELECT pricing_id,
       room_feature,
       ml_generate_embedding_result AS vector_embedding
  FROM ML.GENERATE_EMBEDDING(MODEL `data-connect-demo2.cleaned_data.text_embedding_005`,
                            (SELECT pricing_id, room_feature, room_feature AS content FROM room_features),
                             STRUCT(TRUE AS flatten_json_output,
                                    'SEMANTIC_SIMILARITY' as task_type,
                                    768 AS output_dimensionality));

### <img src="https://storage.googleapis.com/data-analytics-golden-demo/colab-diagrams/BigQuery-32-color.svg" width="30" valign="top" alt="BigQuery"> Search Vector Embeddings

In [ ]:
%%bigquery
----------------------------------------------------------------------------------------------------------------
-- Search: balcony
----------------------------------------------------------------------------------------------------------------
SELECT DISTINCT query.query AS search_string,
       base.room_feature,
       distance
  FROM VECTOR_SEARCH(TABLE `data-connect-demo2.cleaned_data.pricing_embeddings`,
                    'vector_embedding', -- column in table to search
                    (SELECT ml_generate_embedding_result,
                            content AS query
                       FROM ML.GENERATE_EMBEDDING(MODEL `data-connect-demo2.cleaned_data.text_embedding_005`,
                                                 (SELECT 'balcony' AS content),
                                                  STRUCT(TRUE AS flatten_json_output,
                                                        'SEMANTIC_SIMILARITY' as task_type,
                                                        768 AS output_dimensionality) -- struct
                     )),
        top_k => 100)
 WHERE distance < .66
ORDER BY distance;

In [ ]:
%%bigquery
----------------------------------------------------------------------------------------------------------------
-- Search: hair drying
----------------------------------------------------------------------------------------------------------------
SELECT DISTINCT query.query AS search_string,
       base.room_feature,
       distance
  FROM VECTOR_SEARCH(TABLE `data-connect-demo2.cleaned_data.pricing_embeddings`,
                    'vector_embedding', -- column in table to search
                    (SELECT ml_generate_embedding_result,
                            content AS query
                       FROM ML.GENERATE_EMBEDDING(MODEL `data-connect-demo2.cleaned_data.text_embedding_005`,
                                                 (SELECT 'hair drying' AS content),
                                                  STRUCT(TRUE AS flatten_json_output,
                                                        'SEMANTIC_SIMILARITY' as task_type,
                                                        768 AS output_dimensionality) -- struct
                     )),
        top_k => 100)
 WHERE distance < .66
ORDER BY distance;

### <img src="https://storage.googleapis.com/data-analytics-golden-demo/colab-diagrams/BigQuery-32-color.svg" width="30" valign="top" alt="BigQuery"> Compute Competitor Pricing

In [ ]:
%%bigquery
----------------------------------------------------------------------------------------------------------------
-- Find all rooms, using vector embeddings, with and without a "Balcony"
----------------------------------------------------------------------------------------------------------------
SELECT 'Balcony' AS amenity,
       CAST(AVG(pricing.room_size_sqft) AS INT64) AS average_sq_ft,
       CAST(AVG(pricing.price) AS INT64) AS average_price
  FROM VECTOR_SEARCH(TABLE `data-connect-demo2.cleaned_data.pricing_embeddings`,
                    'vector_embedding', -- column in table to search
                    (SELECT ml_generate_embedding_result,
                            content AS query
                       FROM ML.GENERATE_EMBEDDING(MODEL `data-connect-demo2.cleaned_data.text_embedding_005`,
                                                 (SELECT 'balcony' AS content),
                                                  STRUCT(TRUE AS flatten_json_output,
                                                        'SEMANTIC_SIMILARITY' as task_type,
                                                        768 AS output_dimensionality) -- struct
                     )),
        top_k => 100) AS vector_table
        INNER JOIN `data-connect-demo2.cleaned_data.pricing` AS pricing
                ON vector_table.base.pricing_id = pricing.pricing_id
 WHERE distance < .66
 UNION ALL
 SELECT 'No Balcony' AS amenity,
       CAST(AVG(pricing.room_size_sqft) AS INT64) AS average_sq_ft,
       CAST(AVG(pricing.price) AS INT64) AS average_price
  FROM VECTOR_SEARCH(TABLE `data-connect-demo2.cleaned_data.pricing_embeddings`,
                    'vector_embedding', -- column in table to search
                    (SELECT ml_generate_embedding_result,
                            content AS query
                       FROM ML.GENERATE_EMBEDDING(MODEL `data-connect-demo2.cleaned_data.text_embedding_005`,
                                                 (SELECT 'balcony' AS content),
                                                  STRUCT(TRUE AS flatten_json_output,
                                                        'SEMANTIC_SIMILARITY' as task_type,
                                                        768 AS output_dimensionality) -- struct
                     )),
        top_k => 200) AS vector_table
        INNER JOIN `data-connect-demo2.cleaned_data.pricing` AS pricing
                ON vector_table.base.pricing_id = pricing.pricing_id
 WHERE distance >= .66;

### <img src="https://storage.googleapis.com/data-analytics-golden-demo/colab-diagrams/CloudSQL-32-color.svg" width="30" valign="top" alt="AlloyDB"> AlloyDB: Vector Embeddings

##### Open AlloyDB

- https://pantheon.corp.google.com/alloydb/locations/us-central1/clusters/alloy-db-us-central1/studio?project=data-connect-demo2
- Login
  - Database: postgres
  - User: postgres
  - Password: next25

##### Run this in AlloyDB: Query 1

```
----------------------------------------------------------------------------------------------------------------
-- The competition uses "Terrace" and we use Balcony
----------------------------------------------------------------------------------------------------------------
SELECT DISTINCT room_feature,
       vector_embedding::vector <-> embedding('text-embedding-005', 'Terrace')::vector AS distance
  FROM room_amenity
 ORDER BY distance
 LIMIT 100;
```

##### Run this in AlloyDB: Query 3

```
----------------------------------------------------------------------------------------------------------------
-- Compute our average square feet and price for rooms with and without a Balcony
----------------------------------------------------------------------------------------------------------------
SELECT 'Balcony' AS amenity,
       CAST(AVG(room.square_feet) AS INT) AS average_sq_ft,
       CAST(AVG(room_rate.room_rate) AS INT) AS average_price
  FROM room
       INNER JOIN room_amenity
               ON room.room_id= room_amenity.room_id
              AND room_amenity.room_feature = 'Balcony'
       INNER JOIN room_rate
               ON room_rate.room_id = room.room_id
UNION ALL
SELECT 'No Balcony' AS amenity,
       CAST(AVG(room.square_feet) AS INT) AS average_sq_ft,
       CAST(AVG(room_rate.room_rate) AS INT) AS average_price
  FROM room
       INNER JOIN room_rate
               ON room_rate.room_id = room.room_id
              AND room.room_id NOT IN (SELECT room_id FROM room_amenity WHERE room_feature = 'Balcony');
```

### <img src="https://storage.googleapis.com/data-analytics-golden-demo/colab-diagrams/BigQuery-32-color.svg" width="30" valign="top" alt="BigQuery"> BigQuery / AlloyDB Federated Query

In [ ]:
%%bigquery
----------------------------------------------------------------------------------------------------------------
-- Let's compare prices between our competitor pricing (BigQuery) and our transactional data (AlloyDB)
----------------------------------------------------------------------------------------------------------------
SELECT * FROM EXTERNAL_QUERY
(
'us-central1.alloydb-transaction-data',
'''
-- Compute our average square feet and price for rooms with and without a Balcony
SELECT 'AlloyDB' AS source_system,
       'Balcony' AS amenity,
       CAST(AVG(room.square_feet) AS INT) AS average_sq_ft,
       CAST(AVG(room_rate.room_rate) AS INT) AS average_price
  FROM room
       INNER JOIN room_amenity
               ON room.room_id= room_amenity.room_id
              AND room_amenity.room_feature = 'Balcony'
       INNER JOIN room_rate
               ON room_rate.room_id = room.room_id
UNION ALL
SELECT 'AlloyDB' AS source_system,
       'No Balcony' AS amenity,
       CAST(AVG(room.square_feet) AS INT) AS average_sq_ft,
       CAST(AVG(room_rate.room_rate) AS INT) AS average_price
  FROM room
       INNER JOIN room_rate
               ON room_rate.room_id = room.room_id
              AND room.room_id NOT IN (SELECT room_id FROM room_amenity WHERE room_feature = 'Balcony');
'''
) AS alloydb_data
UNION ALL
SELECT 'BigQuery' AS source_system,
       'Balcony' AS amenity,
       CAST(AVG(pricing.room_size_sqft) AS INT64) AS average_sq_ft,
       CAST(AVG(pricing.price) AS INT64) AS average_price
  FROM VECTOR_SEARCH(TABLE `data-connect-demo2.cleaned_data.pricing_embeddings`,
                    'vector_embedding', -- column in table to search
                    (SELECT ml_generate_embedding_result,
                            content AS query
                       FROM ML.GENERATE_EMBEDDING(MODEL `data-connect-demo2.cleaned_data.text_embedding_005`,
                                                 (SELECT 'balcony' AS content),
                                                  STRUCT(TRUE AS flatten_json_output,
                                                        'SEMANTIC_SIMILARITY' as task_type,
                                                        768 AS output_dimensionality) -- struct
                     )),
        top_k => 100) AS vector_table
        INNER JOIN `data-connect-demo2.cleaned_data.pricing` AS pricing
                ON vector_table.base.pricing_id = pricing.pricing_id
 WHERE distance < .66
 UNION ALL
 SELECT 'BigQuery' AS source_system,
        'No Balcony' AS amenity,
       CAST(AVG(pricing.room_size_sqft) AS INT64) AS average_sq_ft,
       CAST(AVG(pricing.price) AS INT64) AS average_price
  FROM VECTOR_SEARCH(TABLE `data-connect-demo2.cleaned_data.pricing_embeddings`,
                    'vector_embedding', -- column in table to search
                    (SELECT ml_generate_embedding_result,
                            content AS query
                       FROM ML.GENERATE_EMBEDDING(MODEL `data-connect-demo2.cleaned_data.text_embedding_005`,
                                                 (SELECT 'balcony' AS content),
                                                  STRUCT(TRUE AS flatten_json_output,
                                                        'SEMANTIC_SIMILARITY' as task_type,
                                                        768 AS output_dimensionality) -- struct
                     )),
        top_k => 200) AS vector_table
        INNER JOIN `data-connect-demo2.cleaned_data.pricing` AS pricing
                ON vector_table.base.pricing_id = pricing.pricing_id
 WHERE distance >= .66
 ORDER BY source_system, amenity;

### <img src="https://storage.googleapis.com/data-analytics-golden-demo/colab-diagrams/BigQuery-32-color.svg" width="30" valign="top" alt="BigQuery"> BigQuery / <img src="https://storage.googleapis.com/data-analytics-golden-demo/colab-diagrams/VertexAI-32-color.svg" width="30" valign="top" alt="VertexAI"> Vertex AI - Gemini

##### PIP install (only need to run once)

In [ ]:
# PIP Installs
import sys
#!{sys.executable} -m pip install plotly

##### Visualize the data

In [ ]:
# Generate a chart with our data
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# 1. Recreate the DataFrame (from the image)
data = {
    'source_system': ['AlloyDB', 'AlloyDB', 'BigQuery', 'BigQuery'],
    'amenity': ['Balcony', 'No Balcony', 'Balcony', 'No Balcony'],
    'average_sq_ft': [461, 420, 458, 412],
    'average_price': [286, 257, 308, 257]
}
df = pd.DataFrame(data)

# 2. Separate data for AlloyDB and BigQuery
alloydb_data = df[df['source_system'] == 'AlloyDB']
bigquery_data = df[df['source_system'] == 'BigQuery']

# 3. Create the Plotly Figure
fig = make_subplots(specs=[[{"secondary_y": False}]])

# Add traces for AlloyDB (Our Hotel)
fig.add_trace(
    go.Bar(
        x=alloydb_data['amenity'],
        y=alloydb_data['average_price'],
        name='Our Hotel (AlloyDB)',
        marker_color='rgb(53, 106, 228)'  # Choose your colors
    ),
    secondary_y=False,
)

# Add traces for BigQuery (Competitor)
fig.add_trace(
    go.Bar(
        x=bigquery_data['amenity'],
        y=bigquery_data['average_price'],
        name='Competitor Pricing (BigQuery)',
        marker_color='rgb(55, 83, 109)'  # Choose your colors
    ),
    secondary_y=False,
)

# 4. Add the Annotation (Arrow and Text)
fig.add_annotation(
    x='Balcony',  # x-coordinate of the arrow's end (AlloyDB Balcony bar)
    y=295,       # y-coordinate of the arrow's end (slightly above AlloyDB bar)
    ax='Balcony', # x-coordinate of the arrow's start (same as x for a vertical arrow)
    ay=350,       # y-coordinate of the arrow's start (higher up)
    xref='x',
    yref='y',
    axref='x',
    ayref='y',
    text='We are undercharging!',  # Annotation text
    showarrow=True,
    arrowhead=2,   # Style of the arrowhead
    arrowsize=1,
    arrowwidth=2,
    arrowcolor="red",
    font=dict(
        family="Courier New, monospace",
        size=16,
        color="red"  # Text color
        )
)

# 5. Update Layout (Titles, Axis Labels, etc.)
fig.update_layout(
    title_text='Average Hotel Room Price Comparison',
    xaxis_title='Amenity',
    yaxis_title='Average Price',
    barmode='group',  # Group the bars side-by-side
    xaxis_tickangle=-45, # Rotate x-axis labels if needed
    template='plotly_white'  # Use a clean white template

)

fig.show()

##### Use Gemini with Retrieval Augmented Generation (RAG)

In [ ]:
%%bigquery
----------------------------------------------------------------------------------------------------------------
-- Now do a RAG using Gemini to process our data for:
-- I work at a hotel and just imported competitor pricing into BigQuery.
-- I reviewed the data for rooms that have a balcony and matched these to the rooms we have from our AlloyDB system.
-- I need to generate some insights to report to management.
-- Can you generate 2 to 3 sentences as to what we should recommend to management.
--  <context>
--    {"source_system":"AlloyDB","amenity":"Balcony","average_sq_ft":461,"average_price":286}
--  	{"source_system":"AlloyDB","amenity":"No Balcony","average_sq_ft":420,"average_price":257}
--  	{"source_system":"BigQuery","amenity":"Balcony","average_sq_ft":458,"average_price":308}
--  	{"source_system":"BigQuery","amenity":"No Balcony","average_sq_ft":412,"average_price":257}
--  </context>
----------------------------------------------------------------------------------------------------------------
WITH llm_prompt AS
(
  SELECT """
  I work at a hotel and just imported competitor pricing into BigQuery.
  I reviewed the data for rooms that have a balcony and matched these to the rooms we have from our AlloyDB system.
  I need to generate some insights to report up to management.
  Can you generate 2 to 3 sentenaces as to how what we should recommend to management.
  Explain your reasoning in your response.
  <context>
  REPLACE-ME-WITH-EMBEDDING-SEARCH-RESULTS
  </context>""" AS prompt
)
, federated_query AS
(
  SELECT source_system,amenity,average_sq_ft,average_price
    FROM EXTERNAL_QUERY('us-central1.alloydb-transaction-data',
          '''
          -- Compute our average square feet and price for rooms with and without a Balcony
          SELECT 'AlloyDB' AS source_system,
                'Balcony' AS amenity,
                CAST(AVG(room.square_feet) AS INT) AS average_sq_ft,
                CAST(AVG(room_rate.room_rate) AS INT) AS average_price
            FROM room
                INNER JOIN room_amenity
                        ON room.room_id= room_amenity.room_id
                        AND room_amenity.room_feature = 'Balcony'
                INNER JOIN room_rate
                        ON room_rate.room_id = room.room_id
          UNION ALL
          SELECT 'AlloyDB' AS source_system,
                'No Balcony' AS amenity,
                CAST(AVG(room.square_feet) AS INT) AS average_sq_ft,
                CAST(AVG(room_rate.room_rate) AS INT) AS average_price
            FROM room
                INNER JOIN room_rate
                        ON room_rate.room_id = room.room_id
                        AND room.room_id NOT IN (SELECT room_id FROM room_amenity WHERE room_feature = 'Balcony');
          ''') AS alloydb_data
  UNION ALL
  SELECT 'BigQuery' AS source_system,
        'Balcony' AS amenity,
        CAST(AVG(pricing.room_size_sqft) AS INT64) AS average_sq_ft,
        CAST(AVG(pricing.price) AS INT64) AS average_price
    FROM VECTOR_SEARCH(TABLE `data-connect-demo2.cleaned_data.pricing_embeddings`,
                      'vector_embedding', -- column in table to search
                      (SELECT ml_generate_embedding_result,
                              content AS query
                        FROM ML.GENERATE_EMBEDDING(MODEL `data-connect-demo2.cleaned_data.text_embedding_005`,
                                                  (SELECT 'balcony' AS content),
                                                    STRUCT(TRUE AS flatten_json_output,
                                                          'SEMANTIC_SIMILARITY' as task_type,
                                                          768 AS output_dimensionality) -- struct
                      )),
          top_k => 100) AS vector_table
          INNER JOIN `data-connect-demo2.cleaned_data.pricing` AS pricing
                  ON vector_table.base.pricing_id = pricing.pricing_id
  WHERE distance < .66
  UNION ALL
  SELECT 'BigQuery' AS source_system,
         'No Balcony' AS amenity,
         CAST(AVG(pricing.room_size_sqft) AS INT64) AS average_sq_ft,
         CAST(AVG(pricing.price) AS INT64) AS average_price
    FROM VECTOR_SEARCH(TABLE `data-connect-demo2.cleaned_data.pricing_embeddings`,
                      'vector_embedding', -- column in table to search
                      (SELECT ml_generate_embedding_result,
                              content AS query
                        FROM ML.GENERATE_EMBEDDING(MODEL `data-connect-demo2.cleaned_data.text_embedding_005`,
                                                  (SELECT 'balcony' AS content),
                                                    STRUCT(TRUE AS flatten_json_output,
                                                          'SEMANTIC_SIMILARITY' as task_type,
                                                          768 AS output_dimensionality) -- struct
                      )),
          top_k => 200) AS vector_table
          INNER JOIN `data-connect-demo2.cleaned_data.pricing` AS pricing
                  ON vector_table.base.pricing_id = pricing.pricing_id
  WHERE distance >= .66
  ORDER BY source_system, amenity
)
, embeddings_data AS
(
  SELECT TO_JSON_STRING(STRUCT(source_system, amenity, average_sq_ft, average_price)) AS embeddings_json
    FROM federated_query
)
, embeddings_array AS
(
SELECT ARRAY_AGG(embeddings_json) AS embeddings_json_array
  FROM embeddings_data
)
SELECT ml_generate_text_result.candidates[0].content.parts[0].text as llm_result
  FROM ML.GENERATE_TEXT(MODEL`cleaned_data.gemini_2_0_flash`,
                       (SELECT REPLACE(prompt,
                                       'REPLACE-ME-WITH-EMBEDDING-SEARCH-RESULTS',
                                       ARRAY_TO_STRING(embeddings_json_array, '\n')) AS prompt
                          FROM llm_prompt CROSS JOIN embeddings_array),
            STRUCT(
              1.0 AS temperature,
              2048 AS max_output_tokens,
              0.95 AS top_p,
              40 AS top_k)
              )

**<font color='#4285f4'>Example Result:</font>**

Here's a recommendation for management based on the competitor pricing data, along with the reasoning:

**Recommendation:**
We should consider slightly increasing the price of our rooms with balconies to better align with competitor pricing, as they are charging a premium for balcony rooms. We should also consider highlighting the slightly larger average square footage of our balcony rooms as a value add when compared to competitors.

**Reasoning:**
The data shows competitors are charging an average of \$308 for balcony rooms, while we are charging $286. This suggests an opportunity to increase revenue by capitalizing on the value customers place on having a balcony. Additionally, our balcony rooms average slightly larger square footage (461 sq ft) compared to the competition (458 sq ft). Highlighting this difference can help justify a price increase and attract customers seeking more spacious accommodations.